In [1]:
#import findspark
#findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
spark = SparkSession.builder.appName('LVM_stroke_stroke').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/13 05:41:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = spark.read.csv("./database/df_con.csv",inferSchema=True,header=True)
#data.columns

In [3]:
my_cols = data.select(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_status',
 'stroke', 'age_grade'])

In [4]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [5]:
assembler = VectorAssembler(inputCols=['hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'age_grade'],outputCol='features')

In [6]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline

In [7]:
lsvm = LinearSVC(featuresCol='features',labelCol='stroke')

In [8]:
pipeline = Pipeline(stages=[assembler,lsvm])

In [9]:
train_data, test_data = data.randomSplit([0.7,.3])

In [10]:
fit_model = pipeline.fit(train_data)

22/10/13 05:42:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv
22/10/13 05:42:32 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


In [11]:
results = fit_model.transform(test_data)

In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='stroke')

In [13]:
results.select('stroke','prediction').show()

+------+----------+
|stroke|prediction|
+------+----------+
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
|     1|       0.0|
+------+----------+
only showing top 20 rows



22/10/13 05:43:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


In [14]:
AUC = my_eval.evaluate(results)

AUC

22/10/13 05:43:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


0.5

In [15]:
npre    = results.filter(results['stroke']==1).filter(results['prediction']==1.0).count()
nstroke = results.filter(results['stroke']==1).count()
npre/nstroke

22/10/13 05:43:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv
22/10/13 05:43:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , id, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id, age_grade
 Schema: _c0, id1, age, hypertension, heart_disease, avg_glucose_level, bmi, smoking_status, stroke, id9, age_grade
Expected: _c0 but found: 
CSV file: file:///home/ubuntu/database/df_con.csv


0.0